In [2]:
import pygal
import pandas as pd
import streamlit as st
from graficos.Tribuna_do_Norte import tnPortal

# Função para tentar múltiplos formatos
def try_parsing_date(text):
    for fmt in ("%Y-%m-%dT%H:%M:%S.%fZ", "%Y-%m-%d %H:%M:%S"):
        try:
            return pd.to_datetime(text, format=fmt)
        except ValueError:
            continue
    return pd.NaT

'''RECEBENDO OS DF'''
# Recebe a tabela com as notícias do impresso
# low_memory=False por a tabela ser grande
df_noticias_impresso = pd.read_csv('tabelas/impresso/dados_impresso.csv', low_memory=False)
df_noticias_impresso['data'] = pd.to_datetime(df_noticias_impresso['data'].apply(try_parsing_date))#VERIFICAR

df_noticias_impresso = df_noticias_impresso.dropna(subset=['data']) #VERIFICAR

# Recebe a tabela com as notícias do impresso
# low_memory=False por a tabela ser grande
df_editorias_impresso = pd.read_csv('tabelas/impresso/EDI_impresso.csv', low_memory=False)
df_editorias_impresso['data'] = pd.to_datetime(df_editorias_impresso['data'].apply(try_parsing_date))#VERIFICAR

df_editorias_impresso = df_editorias_impresso.dropna(subset=['data'])#VERIFICAR

In [3]:
def filtroDeDatasImpresso(start_date, end_date):
    
    df_noticias_impresso_FILTRADAS = df_noticias_impresso.copy()

    #df_noticias_impresso_FILTRADAS['data'] = pd.to_datetime(df_noticias_impresso_FILTRADAS['data']).dt.strftime('%m-%d-%y')

    df_NOTICIAS_impresso_filtrado = df_noticias_impresso_FILTRADAS.loc[(df_noticias_impresso_FILTRADAS['data'] > start_date) & (df_noticias_impresso_FILTRADAS['data'] < end_date)]

    # Altera o formato da data para '%d-%m-%y' após o filtro
    #df_NOTICIAS_impresso_filtrado['data'] = pd.to_datetime(df_NOTICIAS_impresso_filtrado['data'], format='mixed').dt.strftime('%d-%m-%y')
    
    
    
    df_editorias_impresso_FILTRADAS = df_editorias_impresso.copy()

    #df_editorias_impresso_FILTRADAS['data'] = pd.to_datetime(df_editorias_impresso_FILTRADAS['data']).dt.strftime('%m-%d-%y')

    df_editorias_impresso_filtrado = df_editorias_impresso_FILTRADAS.loc[(df_editorias_impresso_FILTRADAS['data'] > start_date) & (df_editorias_impresso_FILTRADAS['data'] < end_date)]

    # Altera o formato da data para '%d-%m-%y' após o filtro
    #df_editorias_impresso_filtrado['data'] = pd.to_datetime(df_editorias_impresso_filtrado['data'], format='mixed').dt.strftime('%d-%m-%y')
    
    
    
    # Recebe o Series df_noticias_impresso['reporter_fotografo'] do df
    # value_counts().reset_index() conta a freqeuncia de cada informação na coluna de reporter_fotografo e organiza em ordem decrescente de acordo com a coluna que conta a freqência de cada informação
    # .replace() renomea linhas da coluna reporter_fotografo
    reporteres_impresso = df_NOTICIAS_impresso_filtrado['reporter_fotografo'].value_counts().reset_index().replace({'icarocesarcarvalho':'Icaro Cesar Carvalho', 'Fernandasouzajh': 'Fernanda Souza', 'adenilson_costa': 'Adenilson Costa'})

    # Renomeando as colunas por conta do .reset_index().replace que criou uma nova com as contagens de cada linha de reporter_fotografo
    reporteres_impresso.columns = ['reporter_fotografo', 'freq']

    # Recebe a coluna 'editoria' do df df_editorias_impresso
    noticias_edi = df_editorias_impresso_filtrado[['editoria', 'freq_edi']]
    
    editorias_impresso = noticias_edi['editoria'].value_counts().reset_index()
    
    editorias_impresso.columns = ['editorias', 'freq']

    # Nesse caso vai sem ['editoria'] por ser um tipo Series e só ter aquela coluna
    noticias_edi_uniRow = noticias_edi.drop_duplicates()

    # Agrupa o DataFrame pelo valor da coluna 'editoria' e soma os valores em cada grupo
    noticias_edi_somado = noticias_edi.drop_duplicates().groupby('editoria')['freq_edi'].sum().reset_index()

    # Foi feita da forma acima por a API do Trello já tinha a contagem de cada editoria disponível e, nesse caso, o calor não estaria correto se fosse usado .value_counts()

    # conta as aparições da linha Pauta sem editoria em editoria
    if 'Pauta sem editoria' in df_editorias_impresso_filtrado['editoria'].values:
        freq_pauta_sem_editoria = df_editorias_impresso_filtrado['editoria'].value_counts()['Pauta sem editoria']
    else:
        freq_pauta_sem_editoria = 0  # Ou qualquer valor padrão desejado

    #freq_pauta_sem_editoria = df_editorias_impresso_filtrado['editoria'].value_counts()['Pauta sem editoria']

    # Adiciona uma nova linha com com a contagem de Pauta sem editoria
    noticias_edi_somado.loc[len(noticias_edi_somado)] = ['Pautas sem editorias', freq_pauta_sem_editoria]

    # Organiza em ordem decrescente de acordo com a coluna freq_edi
    noticias_edi_somado = noticias_edi_somado.sort_values(by= 'freq_edi',ascending=False)
    
    #remove a ultima linha
    noticias_edi_somado = noticias_edi_somado[:len(noticias_edi_somado)-1]
    
    return noticias_edi_somado, df_NOTICIAS_impresso_filtrado, reporteres_impresso, noticias_edi_somado, editorias_impresso

In [5]:
data_minima = pd.to_datetime('2024-06-01')
data_maxima = pd.to_datetime(df_noticias_impresso['data'].max()) + pd.DateOffset(days=1)

# Adicionar seletor de períodos na coluna à esquerda
start_date_ = st.sidebar.date_input("Data de início", data_minima, min_value = data_minima, max_value = data_maxima, format="YYYY-MM-DD") #"DD-MM-YYYY"

end_date_ = st.sidebar.date_input("Data de término", data_maxima, min_value = data_minima, max_value = data_maxima, format="YYYY-MM-DD") #+ pd.DateOffset(days=1) #"DD-MM-YYYY"

periodo = end_date_ - start_date_

# Periodo anteior para gráficos com comparações
start_date_b4 = start_date_ - pd.DateOffset(days = periodo.days+1)

end_date_b4 = end_date_ - pd.DateOffset(days = periodo.days+1) + pd.DateOffset(days=1)

start_date = start_date_.strftime('%Y-%m-%d') #'%d-%m-%Y'
end_date = (end_date_+ pd.DateOffset(days=1)).strftime('%Y-%m-%d')

noticias_edi_somado, df_NOTICIAS_impresso_filtrado, reporteres_impresso, noticias_edi_somado, editorias_impresso = filtroDeDatasImpresso(start_date, end_date)

In [6]:
reporteres_impresso

,reporter_fotografo,freq
0,Felipe Salustino,16
1,Bruno Vital,16
2,Gabriela Liberato,14
3,Larissa Duarte,11
4,Icaro Cesar Carvalho,1


In [43]:
import plotly.graph_objects as go
import plotly.express as px

fig = go.Figure(data=[go.Pie(labels=reporteres_impresso['reporter_fotografo'], values=reporteres_impresso['freq'])])
fig.update_traces(textinfo='value+label', hoverinfo='percent',marker=dict(colors=px.colors.sequential.Oranges, line=dict(color='#66533D', width=1.5)))
fig.show()

In [69]:
import plotly.graph_objects as go
import plotly.express as px

reporteres_impresso = reporteres_impresso.sort_values(by='freq', ascending=True)
fig = go.Figure(data=[go.Bar(x=reporteres_impresso['freq'], y=reporteres_impresso['reporter_fotografo'], orientation='h')])
fig.update_traces(marker_color=px.colors.sequential.Oranges, marker_line_color='#66533D', marker_line_width=1.5)
fig.show() #tudo ok falta editar hover